In [51]:
import pandas as pd
import numpy as np

In [39]:
weibo_m = pd.read_csv('./data/Weibo_M.csv')
weibo = pd.read_csv('./data/Weibo.csv')
wechat = pd.read_csv('./data/Wechat.csv')
news_forum = pd.read_csv('./data/News_forum.csv')

In [40]:
weibo_m = weibo_m[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [41]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [42]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [46]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')

In [89]:
data['AREAS AFFECTED'].unique()

array(['Life Comprehend', 'Products/Service', 'Health Information',
       'Ads/Marketing', 'Customer Service', 'Irrelevant Ads',
       'Charity Events', 'Survey/Questions', 'Contact Information', 'Fund',
       'Corporate Brands', 'Sponsored Events', 'Volunteering Activity',
       'Products', 'Website Issues', 'Corporate News', 'Recruitment',
       'life Comprehend', 'Agent Issues', 'Employment', 'Stocks&Earnings',
       'General Mentioned', 'Employee Information', 'sponsored Events',
       'General Info', 'Corporate news', 'Agent'], dtype=object)

In [90]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
           data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [91]:
data.shape

(30894, 2)